# OPTaaS Batching

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. You can get one <a href="mailto:info@mindfoundry.ai">here</a>.

OPTaaS can facilitate parallel computation, where you generate a batch of configurations, pass them to a number of workers to calculate the results, and then store the results to get the next batch of configurations.

## Connect to OPTaaS using your API Key

In [1]:
library(optaas.client)

client <- OPTaaSClient$new("https://optaas.mindfoundry.ai", "Your OPTaaS API Key")

## Create your Task

In [2]:
task <- client$create_task(
    title="Batching Example",
    parameters=list(
        FloatParameter('x', minimum = 0, maximum = 1),
        FloatParameter('y', minimum = 0.1, maximum = 2)
    )
)

## Set up your workers
This is just a simple example of how you would pass a Configuration to a worker and get a Result back. Your process will of course likely be more complex!

In [3]:
library(parallel)

number_of_workers <- 4

scoring_function <- function(configuration) {
    x <- configuration$values[["x"]]
    y <- configuration$values[["y"]]
    Sys.sleep(1)  # Calculating score...
    score <- (x * y) - (x / y)
    Result$new(configuration=configuration, score=score)
}

spin_off_workers_and_get_results <- function(configurations) {
    mclapply(configurations, scoring_function)
}

## Generate the first batch of configurations

In [4]:
configurations <- task$generate_configurations(number_of_workers)
print(paste(lapply(configurations, `[[`, "values")))

[1] "list(x = 0.5, y = 1.05)"     "list(x = 0.25, y = 1.525)"  
[3] "list(x = 0.75, y = 0.575)"   "list(x = 0.375, y = 0.8125)"


## Record results to get the next batch of configurations

The next batch will be the same size as the number of results you record.

In [5]:
for (batch in seq(1, 3)) {
    results <- spin_off_workers_and_get_results(configurations)
    print(paste("Batch", batch))
    print(lapply(results, function(result) { result$score }))
    flush.console()

    configurations <- task$record_results(results)
}

[1] "Batch 1"
[[1]]
[1] 0.04880952

[[2]]
[1] 0.2173156

[[3]]
[1] -0.8730978

[[4]]
[1] -0.156851

[1] "Batch 2"
[[1]]
[1] 1.045734

[[2]]
[1] 0.06385012

[[3]]
[1] -1.640914

[[4]]
[1] -0.5423534

[1] "Batch 3"
[[1]]
[1] 0.5648003

[[2]]
[1] 0.08435553

[[3]]
[1] 1.5

[[4]]
[1] 0.9496137

